<a href="https://colab.research.google.com/github/B20204/Vision-Arcadia-Submission/blob/main/transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Model

Here is our notebook on the transfer learning model , we have used [Mobilenet](https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4) model , we have used tensorflow hub to import the model , as far as the model is concerned its pretty simple , we have just added a final out layer with softmax activation fxn to recive probablity outputs. also since the model requires input in 224 x 224 we load the data in that particular size.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Concatenate, Input,AveragePooling2D
from tensorflow.keras import layers
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
def preprocess(img):
  blur = cv2.GaussianBlur(img, (1,1), 0)
  kernel = np.array([[-1, -1, -1],[-1, 8,-1],[-1, -1, -1]])
  mask = cv2.filter2D(src=blur, ddepth=-1, kernel=kernel)
  return mask

def normalizeImg(img):
  return img/255

In [3]:
book = {
        0:('/content/drive/MyDrive/Gesture_Data/thumbs up/','Thumbs up'),
        1:('/content/drive/MyDrive/Gesture_Data/w','W'),
        2:('/content/drive/MyDrive/Gesture_Data/one','one'),
        3:('/content/drive/MyDrive/Gesture_Data/Ok_gesture','ok'),
        4:('/content/drive/MyDrive/Gesture_Data/V_gesture','V'),
}

numOfGestures = 5
imgSize = 224

In [4]:
x = []
y = []

for j in range(numOfGestures):
  for i in os.listdir(book[j][0]):
    
    img = cv2.imread(os.path.join(book[j][0],i))
    img = cv2.resize(img,(imgSize,imgSize))
    img = normalizeImg(img)
    x.append(img)
    y.append(j)


x = np.array(x)
y = np.array(y)



In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
y_train = tf.one_hot(np.array(y_train).reshape(-1), numOfGestures)
y_test = tf.one_hot(np.array(y_test).reshape(-1), numOfGestures)

In [6]:
X_train.shape

(623, 224, 224, 3)

In [7]:
transferLearningModel = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3)),
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False), 
    
    tf.keras.layers.Dense(numOfGestures, activation='softmax')
])

transferLearningModel.compile(loss='categorical_crossentropy',optimizer="adam", metrics=['accuracy'])
transferLearningModel.fit(x=X_train, y=y_train, epochs=10)

loss,acc = transferLearningModel.evaluate(x=X_test, y=y_test)

Epoch 1/10
20/20 [==============================] - 20s 796ms/step - loss: 1.6642 - accuracy: 0.2937
Epoch 2/10
20/20 [==============================] - 16s 794ms/step - loss: 1.1614 - accuracy: 0.5714
Epoch 3/10
20/20 [==============================] - 16s 778ms/step - loss: 0.9414 - accuracy: 0.6613
Epoch 4/10
20/20 [==============================] - 16s 779ms/step - loss: 0.7759 - accuracy: 0.7640
Epoch 5/10
20/20 [==============================] - 16s 779ms/step - loss: 0.6465 - accuracy: 0.8266
Epoch 6/10
20/20 [==============================] - 16s 784ms/step - loss: 0.5879 - accuracy: 0.8459
Epoch 7/10
20/20 [==============================] - 16s 777ms/step - loss: 0.5100 - accuracy: 0.8732
Epoch 8/10
20/20 [==============================] - 16s 776ms/step - loss: 0.4534 - accuracy: 0.9037
Epoch 9/10
20/20 [==============================] - 16s 777ms/step - loss: 0.4168 - accuracy: 0.9037
Epoch 10/10
3/3 [==============================] - 2s 500ms/step - loss: 0.7392 - accuracy:

In [ ]:
transferLearningModel.save('/content/drive/MyDrive/Gesture_Data/transferLearningModel/tl_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Gesture_Data/transferLearningModel/tl_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Gesture_Data/transferLearningModel/tl_model/assets


In [8]:
model2 = tf.keras.models.load_model('/content/drive/MyDrive/Gesture_Data/transferLearningModel/tl_model')
model2.evaluate(X_test,y_test)

3/3 [==============================] - 2s 495ms/step - loss: 0.5770 - accuracy: 0.8143


[0.5769751071929932, 0.8142856955528259]